In [ ]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("KAM_Lab2")
         .getOrCreate())

sc = spark.sparkContext

In [3]:
spark

In [4]:
given_courses = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier']
                 , [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2']
                 , [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche']
                 , [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo']
                 , [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus']
                 , [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]


In [5]:
list_given_courses = [x[0] for x in given_courses]
print(list_given_courses)
list_lang = list(set([x[1] for x in given_courses]))
print(list_lang)

[23126, 21617, 16627, 11556, 16704, 13702]
['en', 'es', 'ru']


In [6]:
df = spark.read.json('/labs/slaba02/DO_record_per_line.json')

In [7]:
df.count()

28153

In [8]:
df = df.where(df.lang.isin(list_lang))

In [9]:
df.count()

27158

In [10]:
df.rdd.getNumPartitions()

3

In [11]:
df.where(df.id.isin(list_given_courses)).toPandas()

,cat,desc,id,lang,name,provider
0,,La transformación del aula con el Aprendizaje...,11556,es,Aprendizaje Colaborativo by UNID Universidad I...,Udemy
1,6/economics_finance|15/mathematics_statistics_...,Математическая экономика – это набор моделей в...,13702,ru,Математическая экономика,Intuit
2,,"Hazte más empleable, obtén una nueva competen...",16627,es,Aprende Excel: Nivel Intermedio by Alfonso Rin...,Udemy
3,5/computer_science|14/social_sciences,В курсе рассматривается среда программирования...,16704,ru,Программирование на Lazarus,Intuit
4,5/computer_science,An introduction to how computing can be used t...,21617,en,Preparing for the AP* Computer Science A Exam ...,edX
5,,Improve your SASS skill by learning benefits ...,23126,en,Compass - powerful SASS library that makes you...,Udemy


In [12]:
from pyspark.sql.functions import *
df = df.withColumn("desc", regexp_replace(df.desc, "[^\d\w а-яА-Я]", " "))

In [13]:
df.where(df.lang == "ru").limit(5).toPandas()

,cat,desc,id,lang,name,provider
0,5/computer_science,Часть 1 Продвинутые структуры данных Приорит...,46,ru,Дополнительные главы алгоритмов,Computer Science Center
1,5/computer_science,Splay дерево и декартово дерево Хеширование ...,47,ru,Алгоритмы и структуры данных 2,Computer Science Center
2,5/computer_science,Курс посвящ н теоретическим и практическим асп...,48,ru,Технологии хранения и обработки больших объёмо...,Computer Science Center
3,2/biology_life_sciences|5/computer_science,Биоинформатика это быстро растущий раздел co...,49,ru,Алгоритмы в биоинформатике,Computer Science Center
4,5/computer_science|15/mathematics_statistics_a...,Курс знакомит со сложностью вероятностных вычи...,50,ru,Сложность вычислений и основы криптографии,Computer Science Center


In [14]:
from pyspark.ml.feature import Tokenizer, HashingTF
tokenizer = Tokenizer(inputCol="desc", outputCol="words")

In [15]:
df = tokenizer.transform(df)

In [16]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer

In [17]:
stop_words = (StopWordsRemover.loadDefaultStopWords("russian") +
            StopWordsRemover.loadDefaultStopWords("spanish") +
            StopWordsRemover.loadDefaultStopWords("english") + [' ', ''])

In [18]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words_filtered", stopWords=stop_words)

In [19]:
df = swr.transform(df)

In [20]:
df.limit(5).toPandas()

,cat,desc,id,lang,name,provider,words,words_filtered
0,3/business_management|6/economics_finance,This course introduces the basic financial sta...,4,en,Accounting Cycle: The Foundation of Business M...,Canvas Network,"[this, course, introduces, the, basic, financi...","[course, introduces, basic, financial, stateme..."
1,11/law,This online course will introduce you to Ameri...,5,en,American Counter Terrorism Law,Canvas Network,"[this, online, course, will, introduce, you, t...","[online, course, introduce, american, laws, re..."
2,14/social_sciences,We live in a digitally connected world The wa...,7,en,Becoming a Dynamic Educator,Canvas Network,"[we, live, in, a, digitally, connected, world,...","[live, digitally, connected, world, way, infor..."
3,2/biology_life_sciences,This self paced course is designed to show tha...,8,en,Bioethics,Canvas Network,"[this, self, paced, course, is, designed, to, ...","[self, paced, course, designed, show, ethical,..."
4,9/humanities|15/mathematics_statistics_and_dat...,This game based course provides prospective st...,9,en,"College Foundations: Reading, Writing, and Math",Canvas Network,"[this, game, based, course, provides, prospect...","[game, based, course, provides, prospective, s..."


In [21]:
tdf = df.select(df.lang, F.explode(df.words_filtered))

In [22]:
tdf.show()

+----+----------+
|lang|       col|
+----+----------+
|  en|    course|
|  en|introduces|
|  en|     basic|
|  en| financial|
|  en|statements|
|  en|      used|
|  en|businesses|
|  en|      well|
|  en| essential|
|  en|     tools|
|  en|      used|
|  en|   prepare|
|  en|    course|
|  en|     serve|
|  en|  resource|
|  en|      help|
|  en|  business|
|  en|  students|
|  en|   succeed|
|  en|  upcoming|
+----+----------+
only showing top 20 rows



In [23]:
group_user = tdf.groupBy('lang').agg(collect_set('col').alias('words'))

In [24]:
group_user.show()

+----+--------------------+
|lang|               words|
+----+--------------------+
|  en|[samplesample, fl...|
|  es|[exprimiendolinke...|
|  ru|[downhole, рынкиr...|
+----+--------------------+



In [ ]:
df1 = df.select(F.explode(df.words_filtered))

In [ ]:
df1 = df1.groupby(df1.col).count()\
    .select(df1.col.alias("word"), col("count").alias("count_w"))
df1.limit(5).show()

In [ ]:
df1.sort(df1.count_w.desc()).limit(5).show()

In [ ]:
df1 = df1.where((df1.count_w >= 20000) | (df1.count_w == 1) | (F.length(df1.word) < 3)).select(df1.word)

In [ ]:
stop_words = df1.rdd.map(lambda x: str(x[0])).take(df1.count())

In [ ]:
swr = StopWordsRemover(inputCol=swr.getOutputCol(), outputCol="words_filtered_final", stopWords=stop_words)

In [ ]:
df = swr.transform(df)

In [ ]:
df.where(df.id.isin(list_given_courses)).toPandas()

In [ ]:
hasher_freq = HashingTF(numFeatures=10000, binary=False, inputCol=swr.getOutputCol(), outputCol="word_vector_freq")
dataset2_freq = hasher_freq.transform(df)

In [ ]:
dataset2_freq.limit(5).toPandas()

In [ ]:
from pyspark.ml.feature import Normalizer
normalizer = Normalizer(inputCol=hasher_freq.getOutputCol(), outputCol="norm")

In [ ]:
dataset2_freq = normalizer.transform(dataset2_freq)

In [ ]:
df_variant = dataset2_freq.where(dataset2_freq.id.isin(list_given_courses))\
        .select("id", "name", "lang", "norm")

In [ ]:
df_check = dataset2_freq.where(~dataset2_freq.id.isin(list_given_courses))\
        .select("id", "name", "lang", "norm")

In [ ]:
df_variant.count()

In [ ]:
df_check.count()

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
# cosine similarity
dot_udf = udf(lambda x,y: float(x.dot(y)), DoubleType())

result = df_variant.alias("i")\
    .join(df_check.alias("j"), col("i.lang") == col("j.lang"))\
    .select(
        col("i.id").alias("id"), 
        col("j.id").alias("recomended_id"),
        col("j.name").alias("name"),
        dot_udf("i.norm", "j.norm").alias("dot")
    )\
    .sort(col("id"), col("dot").desc(), col("name"), col("recomended_id"))

result.show(10)

In [ ]:
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("id").orderBy(col("id"), col("dot").desc(), col("name"), col("recomended_id"))
result = result.withColumn("row_number", row_number().over(windowSpec)).filter(col("row_number") <= 10)

In [ ]:
result_driver = result.select(col("id"), col("recomended_id")).collect()

In [ ]:
result_driver_json = {}
for x in result_driver:
    if x.id not in result_driver_json:
        result_driver_json[x.id] = []
    result_driver_json[x.id].append(x.recomended_id)
result_driver_json

In [ ]:
import json
with open('lab02.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(result_driver_json))

In [ ]:
[[11556, 0.7], [13702, 0.3], [16627, 0.5], [16704, 0.3], [21617, 0.1], [23126, 0.1]]
[[11556, 0.7], [13702, 0.7], [16627, 0.6], [16704, 0.6], [21617, 0.6], [23126, 0.3]]

In [ ]:
spark.stop()